In [22]:
# Deep Learning CNN model to recognize face
TrainingImagePath='/content/drive/MyDrive/Face-Images/Face Images/Final Training Images/'
TestingImagePath='/content/drive/MyDrive/Face-Images/Face Images/Final Testing Images/'

from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
        shear_range=0.1,
        zoom_range=0.1,
        horizontal_flip=True)

# Defining pre-processing transformations on raw images of testing data
# No transformations are done on the testing images
test_datagen = ImageDataGenerator()

# Generating the Training Data
training_set = train_datagen.flow_from_directory(
        TrainingImagePath,
        target_size=(64, 64),
        batch_size=16,
        class_mode='categorical')


# Generating the Testing Data
test_set = test_datagen.flow_from_directory(
        TestingImagePath,
        target_size=(64, 64),
        batch_size=16,
        class_mode='categorical')

# Printing class labels for each face
test_set.class_indices

Found 244 images belonging to 16 classes.
Found 64 images belonging to 16 classes.


{'face1': 0,
 'face10': 1,
 'face11': 2,
 'face12': 3,
 'face13': 4,
 'face14': 5,
 'face15': 6,
 'face16': 7,
 'face2': 8,
 'face3': 9,
 'face4': 10,
 'face5': 11,
 'face6': 12,
 'face7': 13,
 'face8': 14,
 'face9': 15}

In [23]:
test_set.class_indices.keys()

dict_keys(['face1', 'face10', 'face11', 'face12', 'face13', 'face14', 'face15', 'face16', 'face2', 'face3', 'face4', 'face5', 'face6', 'face7', 'face8', 'face9'])

In [24]:
'''############ Creating lookup table for all faces ############'''
# class_indices have the numeric tag for each face
TrainClasses=training_set.class_indices

# Storing the face and the numeric tag for future reference
ResultMap={}
for faceValue,faceName in zip(TrainClasses.values(),TrainClasses.keys()):
    ResultMap[faceValue]=faceName

# Saving the face map for future reference
import pickle
with open("/content/drive/MyDrive/Face-Images/Face Images/ResultsMap.pkl", 'wb') as fileWriteStream:
    pickle.dump(ResultMap, fileWriteStream)

# The model will give answer as a numeric tag
# This mapping will help to get the corresponding face name for it
print("Mapping of Face and its ID",ResultMap)

# The number of neurons for the output layer is equal to the number of faces
OutputNeurons=len(ResultMap)
print('\n The Number of output neurons: ', OutputNeurons)

Mapping of Face and its ID {0: 'face1', 1: 'face10', 2: 'face11', 3: 'face12', 4: 'face13', 5: 'face14', 6: 'face15', 7: 'face16', 8: 'face2', 9: 'face3', 10: 'face4', 11: 'face5', 12: 'face6', 13: 'face7', 14: 'face8', 15: 'face9'}

 The Number of output neurons:  16


In [26]:
'''######################## Create CNN deep learning model ########################'''
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPool2D
from keras.layers import Flatten
from keras.layers import Dense
 
'''Initializing the Convolutional Neural Network'''
classifier= Sequential()
 
''' STEP--1 Convolution
# Adding the first layer of CNN
# we are using the format (64,64,3) because we are using TensorFlow backend
# It means 3 matrix of size (64X64) pixels representing Red, Green and Blue components of pixels
'''
classifier.add(Convolution2D(32, kernel_size=(5, 5), strides=(1, 1), input_shape=(64,64,3), activation='relu'))
 
'''# STEP--2 MAX Pooling'''
classifier.add(MaxPool2D(pool_size=(2,2)))
 
'''############## ADDITIONAL LAYER of CONVOLUTION for better accuracy #################'''
classifier.add(Convolution2D(64, kernel_size=(5, 5), strides=(1, 1), activation='relu'))
 
classifier.add(MaxPool2D(pool_size=(2,2)))
 
'''# STEP--3 FLattening'''
classifier.add(Flatten())
 
'''# STEP--4 Fully Connected Neural Network'''
classifier.add(Dense(64, activation='relu'))
 
classifier.add(Dense(OutputNeurons, activation='softmax'))
 
'''# Compiling the CNN'''
#classifier.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
classifier.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics=["accuracy"])
 
###########################################################
import time
# Measuring the time taken by the model to train
StartTime=time.time()
 
# Starting the model training
classifier.fit_generator(
                    training_set,
                    steps_per_epoch=10,
                    epochs=10)
 
EndTime=time.time()
print("###### Total Time Taken: ", round((EndTime-StartTime)/60), 'Minutes ######')


Epoch 1/10


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:47: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


10/10 [==============================] - 3s 203ms/step - loss: 40.1482 - accuracy: 0.0676
Epoch 2/10
10/10 [==============================] - 2s 221ms/step - loss: 2.9029 - accuracy: 0.0375
Epoch 3/10
10/10 [==============================] - 2s 202ms/step - loss: 2.7481 - accuracy: 0.0473
Epoch 4/10
10/10 [==============================] - 2s 205ms/step - loss: 2.6389 - accuracy: 0.1216
Epoch 5/10
10/10 [==============================] - 2s 199ms/step - loss: 2.4049 - accuracy: 0.2162
Epoch 6/10
10/10 [==============================] - 2s 208ms/step - loss: 2.0486 - accuracy: 0.3716
Epoch 7/10
10/10 [==============================] - 2s 203ms/step - loss: 1.5231 - accuracy: 0.4730
Epoch 8/10
10/10 [==============================] - 2s 205ms/step - loss: 0.9136 - accuracy: 0.6892
Epoch 9/10
10/10 [==============================] - 2s 205ms/step - loss: 0.8580 - accuracy: 0.7635
Epoch 10/10
10/10 [==============================] - 2s 202ms/step - loss: 0.4809 - accuracy: 0.8716
###### To

In [27]:
'''########### Making single predictions ###########'''
import numpy as np
from keras.preprocessing import image
import tensorflow as tf
from tensorflow.keras.utils import img_to_array

ImagePath='/content/drive/MyDrive/Face-Images/Face Images/Final Testing Images/face10/1face10.jpg'
#test_image=image.load_img(ImagePath,target_size=(64, 64))
test_image = tf.keras.preprocessing.image.load_img(ImagePath,target_size=(64, 64))
test_image=img_to_array(test_image)

test_image=np.expand_dims(test_image,axis=0)

result=classifier.predict(test_image,verbose=0)
#print(training_set.class_indices)

print('####'*10)
print('Prediction is: ',ResultMap[np.argmax(result)])

########################################
Prediction is:  face10
